# the movement model

M is a matrix of [np, np] and C and N vectors of size [np, m].

Mij = individuals moving for i to j.

1. Colonized ppl moving outside unit $i$. The people moving outside of unit i are $\sum_j M_{ij}$.

    $C_{i}^{out}= \sum_j M_{ij} \frac{C_i}{N_i}$

    In code it will be
    ```
    Cout_i = np.sum(M[i, :]) * C[i, :]/N[i, :]
    ```
    So for putting all the people in a matix it will be
    ```
    Cout = np.sum(M, axis=1, keepdims=True) * C / N
    ```
2. Colonized people moving into unit $i$.
   For a 3 metapop model, the people moving into population $i=1$ will be $M_{21}, M_{31}$
   So the people moving inside colonized are $C_1^{out}=M_{21} * C_2/N_2$ and $C_2^{out}=M_{31} * C_3/N_3$.
    As a matrix multiplication this is $M_{:,1}^T \times C/N$, where $M_{:,1}^T=[M_{11}, M_{21}, M_{31}]$ and $C=[C_1, C_2, C_3]^T$.

   The total people moving inside colonized are $C_i^{out}=\sum_j M_{ji} * C_j/N_j$.

   So, in matrix multiplication (I think - not sure) the people moving into all population can be writted as below, gotta be carefull to make the diagonal in M zero!!! or will add people in the same ward.

   $$C_{in} = M^T \times C$$

In [1]:
import numpy as np


model_settings            = {}
model_settings["m"]       = 100
model_settings["num_pop"] = 2

N0 = np.array([[1e3], [1.5e3]]) * np.ones((model_settings["num_pop"], model_settings["m"]))
C0 = 0.05 * np.ones((model_settings["num_pop"], model_settings["m"]))


m       = model_settings["m"]
num_pop = model_settings["num_pop"]

N       = N0
C       = C0 * N

a2c    = np.zeros((num_pop, m))
inc    = np.zeros((num_pop, m))

x0     = np.array([C, a2c, inc]) # [num_state_vars x num_pop x m]


M = np.random.uniform(size=(num_pop, num_pop))
np.fill_diagonal(M, 0)


Cout = np.sum(M, axis=1, keepdims=True) * np.clip(np.nan_to_num(C/N), 0, 1)
Cin  = M.T @ np.clip(np.nan_to_num(C/N), 0, 1)


moveitojC = np.full((num_pop, num_pop, m), np.nan)

for j in range(num_pop):
    for i in range(num_pop):
        if i==j:
            mij = 0
        else:
            mij = M[i,j]

        moveitojC[i, j,:] = M[i,j] * np.clip(np.nan_to_num(C[i, :] / N[i, :]),0,1)

Cout_loop = np.sum(moveitojC, axis=1, keepdims=False)
Cin_loop  = np.sum(moveitojC, axis=0, keepdims=False)


Cout == Cout_loop, Cin == Cin_loop


(array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  Tr